# Текст в речь на русском языке
[![Открыть в Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/self-destruction/XTTS2_colab/blob/main/XTTS_v2_RU.ipynb)
#### Для преобразования текста в речь необходим небольшой отрывок чистой речи пародируемого. Далее файл автоматически очищается от пауз и шума. И проводится преобразование текста в речь.

In [ ]:
#@title # Установка зависимостей TTS
%cd /content
!git clone https://github.com/coqui-ai/TTS.git -b dev
!mkdir /content/input
!mkdir /content/output

XTTS_V2_DIR = '/content/XTTS-v2'
!mkdir {XTTS_V2_DIR}
!wget https://huggingface.co/coqui/XTTS-v2/resolve/main/model.pth -P {XTTS_V2_DIR} &> /dev/null
!wget https://huggingface.co/coqui/XTTS-v2/resolve/main/config.json -P {XTTS_V2_DIR} &> /dev/null

TTS_DIR = '/content/TTS'
%cd {TTS_DIR}
!apt install -q ffmpeg
!pip install virtualenv
!virtualenv venv
!source {TTS_DIR}/venv/bin/activate; pip3 install -q llvmlite --ignore-installed
!source {TTS_DIR}/venv/bin/activate; pip3 install -q torch torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!source {TTS_DIR}/venv/bin/activate; pip install -q -r requirements.txt
# костыли
!sed -i 's/ # Not sure what sets this to None, but applied a fix to prevent crashing./ return True/g' {TTS_DIR}/TTS/api.py
!sed -i 's/text=sen,/text=sen.rstrip(\'.\'),/g' {TTS_DIR}/TTS/utils/synthesizer.py

In [ ]:
from google.colab import files
import os
from IPython.display import Audio

#@title Загрузка mp3-референса
file_folder = '/content/input' #@param {type:"string"}
!mkdir -p {file_folder}
%cd {file_folder}
INPUT_FILE = ''

uploaded = files.upload()

for file_name in uploaded.keys():
  INPUT_FILE = os.path.join(file_folder, file_name)

print(f"\nПроверь, всё ли ок 👇")
audio = Audio(INPUT_FILE, autoplay=False)
display(audio)

In [ ]:
import locale
from IPython.display import Audio
locale.getpreferredencoding = lambda: "UTF-8"
#@title Предварительная очистка аудио
# @markdown ### Кол-во секунд, которые мы возьмём в чистый файл:
TIME_SEC=3 # @param {type:"slider", min:1, max:20, step:1}
# @markdown ---
# @markdown ### Сдвиг начала:
# @markdown ##### (на случай, если не повезло с началом)
TIME_OFFSET=1 # @param {type:"slider", min:0, max:20, step:1}
# @markdown ---
# @markdown ### Порог шума в dB (-25dB - оптимальное значение, -80dB для чистого исходника, -20dB для ОЧЕНЬ грязного):
NOISE_THRESHOLD=-25   # @param {type:"slider", min:-120, max:-1, step:1}

INPUT_CLEAN_FILE = '/content/input/input_audio_clean.mp3'

TIME_SEC=str(TIME_SEC+TIME_OFFSET) # хак при ненулевом сдвиге
NOISE_THRESHOLD=str(NOISE_THRESHOLD)
TIME_OFFSET=str(TIME_OFFSET)

!ffmpeg -y -hide_banner -loglevel error -i '{INPUT_FILE}' -af lowpass=8000,highpass=75,areverse,silenceremove=stop_periods=-1:stop_duration=1:stop_threshold={NOISE_THRESHOLD}dB:stop_silence=0:start_periods=9000:start_silence=0,areverse,silenceremove=stop_periods=-1:stop_duration=1:stop_threshold={NOISE_THRESHOLD}dB:stop_silence=0:start_periods=9000:start_silence=0,atrim={TIME_OFFSET}:{TIME_SEC} {INPUT_CLEAN_FILE}
print(f"\nПроверь, всё ли ок 👇")
audio = Audio(INPUT_CLEAN_FILE, autoplay=False)
display(audio)

In [ ]:
%cd {TTS_DIR}
locale.getpreferredencoding = lambda: "UTF-8"
#@title # 🚀 Генерация Текста в речь
# @markdown ### Текст:
PROMPT = ""  #@param {type:"string"}
# @markdown ---
# @markdown ### Путь к результату:
OUTPUT_FILE = "/content/output/speech.mp3"  #@param {type:"string"}

from IPython.display import Audio, display, HTML, FileLink;

cmd = (
    f"import torch; from TTS.api import TTS; "
    f"device = 'cuda' if torch.cuda.is_available() else 'cpu'; "
    f"tts = TTS(model_path='/content/XTTS-v2', config_path='/content/XTTS-v2/config.json', progress_bar=False).to(device); "
    f"tts.tts_to_file(text='{PROMPT}', speaker_wav='{INPUT_CLEAN_FILE}', language='ru', file_path='{OUTPUT_FILE}'); "
)

!source {TTS_DIR}/venv/bin/activate; python -c "{cmd}"

audio = Audio(OUTPUT_FILE, autoplay=False)
display(audio)